# Baseline CNN models for Galaxy challenge - renormalized data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

plt.rcParams["axes.grid"] = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data

In [ ]:
sigma = 0.0125

Use normalized data

In [ ]:
train_images, train_ns = torch.load(f"../data/315/dataset_train_renorm_{sigma}.pt")
test_images, test_ns = torch.load(f"../data/315/dataset_test_renorm_{sigma}.pt")
test_images_kaggle, test_ns_kaggle = torch.load(f"../data/315/dataset_kaggle_renorm_{sigma}.pt")

In [ ]:
(
    train_images.shape,
    train_ns.shape,
    test_images.shape,
    test_ns.shape,
    test_images_kaggle.shape,
    test_ns_kaggle.shape,
)

In [ ]:
train_images = train_images.unsqueeze(1)
test_images = test_images.unsqueeze(1)
test_images_kaggle = test_images_kaggle.unsqueeze(1)

In [ ]:
train_images.shape, train_ns.shape

In [ ]:
train_dataset = TensorDataset(train_images, train_ns)
test_dataset = TensorDataset(test_images, test_ns)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
kaggle_dataset = TensorDataset(test_images_kaggle, test_ns_kaggle)

In [ ]:
# NOTE: don't shuffle for the kaggle dataset
kaggle_loader = DataLoader(kaggle_dataset, batch_size=32, shuffle=False, num_workers=2)

## Display

In [ ]:
i = torch.randint(1, 2500)
plt.imshow(train_images[i].squeeze(), cmap="gray")
plt.title(f"{train_ns[i]}")

In [ ]:
i = torch.randint(1, 2500)
plt.imshow(test_images[i].squeeze(), cmap="gray")
plt.title(f"{test_ns[i]}");

In [ ]:
i = torch.randint(1, 2500)
plt.imshow(test_images_kaggle[i].squeeze(), cmap="gray")
plt.title(f"{test_ns_kaggle[i]}");

# Utils

In [ ]:
dim = 50

In [ ]:
n_classes = 7

In [ ]:
num_epochs = 40
lr = 1e-3

In [ ]:
def train_model(model, train_loader, test_loader, num_epochs, lr, model_save_path):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    test_losses = []

    pbar = tqdm(
        total=num_epochs,
        desc=f"Epoch: 0/{num_epochs} | Train Loss: N/A | Test Loss: N/A",
    )

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        test_running_loss = 0.0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_running_loss += loss.item()

        test_loss = test_running_loss / len(test_loader)
        test_losses.append(test_loss)

        pbar.set_description(
            f"Epoch: {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} "
            f"| Test Loss: {test_loss:.4f}"
        )
        pbar.update(1)

        if (epoch + 1) % 10 == 0:
            checkpoint_path = f"{model_save_path}_epoch_{epoch + 1}.pth"
            torch.save(model.state_dict(), checkpoint_path)

    return train_losses, test_losses

# VGGs

In [ ]:
from torchvision import models

## VGG11

In [ ]:
def make_vgg11(device):
    model = models.vgg11_bn(pretrained=False)

    # Need the first conv layer to accept 1 channel instead of 3.
    model.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)

    model.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    # Modify the classifier from CIFAR10
    model.classifier = nn.Sequential(
        nn.Linear(512, 256), nn.ReLU(inplace=True), nn.Dropout(0.5), nn.Linear(256, 7)
    )

    return model.to(device)

In [ ]:
model = make_vgg11(device)

In [ ]:
train_losses, test_losses = train_model(
    model,
    train_loader,
    test_loader,
    num_epochs,
    lr=1e-3,
    model_save_path=f"../res/models/galaxy_challenge/vgg11_renorm_{sigma}",
)

## $\sigma = 0.01$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

In [ ]:
best_model = make_vgg11(device)
best_model.load_state_dict(
    torch.load(
        f"../res/models/galaxy_challenge/vgg11_{sigma}_epoch_10.pth",
        map_location=device,
    )
)

In [ ]:
best_model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

Generate a test submission:

In [ ]:
best_model.eval()

predictions = []

with torch.no_grad():
    for images in kaggle_loader:
        images = images.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

In [ ]:
image_ids = [f"{i}" for i in range(test_ns_kaggle.shape[0])]
prediction_df = pd.DataFrame({"id": image_ids, "label": predictions})

In [ ]:
prediction_df.to_csv(f"../data/315/kaggle/kaggle_{sigma}_test_predictions.csv", index=False)

## $\sigma = 0.0125$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.015$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

In [ ]:
best_model = make_vgg11(device)
best_model.load_state_dict(
    torch.load(
        f"../res/models/galaxy_challenge/vgg11_renorm_{sigma}_epoch_30.pth",
        map_location=device,
    )
)

In [ ]:
best_model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.02$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

In [ ]:
best_model = make_vgg11(device)
best_model.load_state_dict(
    torch.load(
        f"../res/models/galaxy_challenge/vgg11_renorm_{sigma}_epoch_20.pth",
        map_location=device,
    )
)

In [ ]:
best_model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.1$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

# ChatGPT

In [ ]:
class StarCounterCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Input shape: (batch_size, 1, 50, 50)
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=16, kernel_size=3, padding=1
        )  # (16, 50, 50)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # (16, 25, 25)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # (32, 25, 25)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # (32, 12, 12)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # (64, 12, 12)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # (64, 6, 6)

        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 7)  # 7 output classes for 0-6 stars

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = self.pool3(x)

        x = x.view(x.size(0), -1)  # flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
model = StarCounterCNN()
model = model.to(device)

In [ ]:
train_losses, test_losses = train_model(
    model,
    train_loader,
    test_loader,
    num_epochs,
    lr,
    f"../res/models/galaxy_challenge/cgpt_renorm_{sigma}",
)

## $\sigma = 0.01$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

In [ ]:
best_model = StarCounterCNN().to(device)
best_model.load_state_dict(
    torch.load(f"../res/models/galaxy_challenge/cgpt_{sigma}_epoch_20.pth", map_location=device)
)

In [ ]:
best_model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.0125$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.015$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

In [ ]:
best_model = StarCounterCNN().to(device)
best_model.load_state_dict(
    torch.load(
        f"../res/models/galaxy_challenge/cgpt_renorm_{sigma}_epoch_20.pth",
        map_location=device,
    )
)

In [ ]:
best_model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.02$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

In [ ]:
best_model = StarCounterCNN().to(device)
best_model.load_state_dict(
    torch.load(
        f"../res/models/galaxy_challenge/cgpt_renorm_{sigma}_epoch_20.pth",
        map_location=device,
    )
)

In [ ]:
best_model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.1$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

# Gemini

In [ ]:
class StarClassifier(nn.Module):
    def __init__(self, num_classes=7):  # 0 to 6 stars, so 7 classes
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate the flattened size after max pooling
        # Input size: 50x50
        # After maxpool1: 25x25
        # After maxpool2: 12x12 (or 13x13 depending on rounding)
        # After maxpool3: 6x6 (or 7x7 depending on rounding)

        self.fc1 = nn.Linear(64 * 6 * 6, 128)  # adjust if necessary if 7x7 is used.
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = self.maxpool3(self.relu3(self.conv3(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu4(self.fc1(x))
        return self.fc2(x)


# Example usage:
# model = StarClassifier()
# input_tensor = torch.randn(1, 1, 50, 50)  # Batch size 1, 1 channel (grayscale), 50x50 image
# output = model(input_tensor)
# print(output.shape) # Should be [1,7]

In [ ]:
model = StarClassifier()
model.to(device)

In [ ]:
train_losses, test_losses = train_model(
    model,
    train_loader,
    test_loader,
    num_epochs,
    lr,
    f"../res/models/galaxy_challenge/gemini_renorm_{sigma}",
)

## $\sigma = 0.01$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

In [ ]:
best_model = StarClassifier().to(device)
best_model.load_state_dict(
    torch.load(
        f"../res/models/galaxy_challenge/gemini_{sigma}_epoch_10.pth",
        map_location=device,
    )
)

In [ ]:
best_model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.0125$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.015$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.02$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

In [ ]:
best_model = StarClassifier().to(device)
best_model.load_state_dict(
    torch.load(
        f"../res/models/galaxy_challenge/gemini_renorm_{sigma}_epoch_10.pth",
        map_location=device,
    )
)

In [ ]:
best_model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")

## $\sigma = 0.1$

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(test_losses, label="test")
plt.legend();

In [ ]:
model.eval()
correct = 0
total = 0

total_kaggle = 0
correct_kaggle = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    for images, labels in kaggle_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_kaggle += labels.size(0)
        correct_kaggle += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
print(f"Kaggle Accuracy: {100 * correct_kaggle / total_kaggle:.2f}%")